In [7]:
from datetime import tzinfo
import datetime as dt
from scipy.io import loadmat
import pandas as pd
import numpy as np

file_name = "../../../data/visible_events.mat"
data = loadmat(file_name)
col = ['distance', 'depth', 'phase', 'travel_time', 'year', 'month', 'day', 'h', 'm', 's', 'mag', 'e1', 'e2', 'e3', 'associated_file' ]
df = []
for line in data['events_list'].flatten():
    # print(line.split())
    df.append(line.split())
df = np.array(df)
df = pd.DataFrame(df,columns=col)
df.astype = {'distance'       :   'float64',
             'depth'          :   'float64',
             'phase'          :   'str',
             'travel_time'    :   'float64',
             'year'           :   'float64',
             'month'          :   'float64',
             'day'            :   'float64',
             'h'              :   'float64',
             'm'            :   'float64',
             's'              :   'float64',
             'mag'            :   'float64',
             'e1'             :   'float64',
             'e2'             :   'float64',
             'e3'             :   'float64',
             'associated_file':   'str' }
df['datetime'] = pd.to_datetime(df[['year','month','day','h','m','s']],utc=True)
df = df[['datetime','distance','depth','phase','travel_time','mag','associated_file']]
df["phase"].unique()

array(['PKIKP', 'PKP', 'P', 'PKiKP'], dtype=object)

In [8]:
import datetime
from haversine import haversine, Unit
from obspy.taup import TauPyModel
######################################
PATH = 'C:/Users/Romain/PycharmProjects/NEIC_ISC_join/data'
NAME_NEIC = '/NEIC_2018_M6.csv'

######################################
######################################
neic = pd.read_csv(PATH + NAME_NEIC, parse_dates=['time'], date_format='ISO8601')

#print(neic.columns.values)
#['time' 'latitude' 'longitude' 'depth' 'mag' 'magType' 'nst' 'gap' 'dmin'
# 'rms' 'net' 'id' 'updated' 'place' 'type' 'horizontalError' 'depthError'
# 'magError' 'magNst' 'status' 'locationSource' 'magSource']
# ELAN coords
st_lat, st_lon = -56.4602,	62.976
coords = (-18.1125,-178.153)
Elan = (st_lat, st_lon)
dist = haversine(coords, Elan) / 111
depth = 600
model = TauPyModel(model="iasp91")
arrivals = model.get_travel_times(source_depth_in_km=depth,
                              distance_in_degree=dist, phase_list=["P"])
print(dist)
print(arrivals[0])

h = datetime.datetime(2018, 8, 19, 00, 19, 40, 67)
h = h + datetime.timedelta(seconds=arrivals[0].time)
h

89.83874757777151
P phase arrival at 715.742 seconds


datetime.datetime(2018, 8, 19, 0, 31, 35, 742051)

In [9]:
from src.utils.data_reading.sound_data.sound_file import DatFile
from src.utils.data_reading.sound_data.sound_file_manager import DatFilesManager

In [10]:
from datetime import timedelta
from utils.physics.signal.make_spectrogram import make_spectrogram
import matplotlib.pyplot as plt
i =14
path = "F:/OHASISBIO/2018/2018_ELAN_raw"  # put a DAT directory here
start = df['datetime'].iloc[i]+timedelta(seconds=np.float64(df['travel_time'].iloc[i]))-timedelta(minutes=5)
start = start.replace(tzinfo=None)
end = df['datetime'].iloc[i]+timedelta(seconds=np.float64(df['travel_time'].iloc[i]))+timedelta(minutes=5)
end =end.replace(tzinfo=None)

manager = DatFilesManager(path)
data = manager.get_segment(start,end)
print(manager.find_file_name(start))
f, t, spectro = make_spectrogram(data, manager.sampling_f, t_res=0.5, f_res=0.5, return_bins=True)
plt.figure()
plt.imshow(spectro, aspect="auto", cmap="jet", extent=(t[0], t[-1], f[0], f[-1]))
plt.axvline(x=300)
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.colorbar()

281


In [11]:
f = manager.sampling_f
n = len(data)
%matplotlib qt
times = np.arange(n)/f
print(len(times))
print(n)
plt.figure()
plt.plot(times, data)
plt.axvline(0.5*n/f, color='r')

143997
143997


In [12]:
import numpy as np
import scipy.signal as signal
import pywt
import matplotlib.pyplot as plt
%matplotlib

# Constants
fs = 240  # Sampling frequency (Hz)
lowcut = 0.06 # Lower cutoff frequency for band-pass filter
highcut = 2  # Upper cutoff frequency for band-pass filter

# Load Your Data (Replace with your own data)
# Example: Simulated seismic signal with noise
t = times  # 60 seconds of data
data = data

# 1. Band-Pass Filter (Butterworth)
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data)
    # a = signal.butter(order, [low, high], btype='band',output='sos')
    # return signal.sosfiltfilt( a, data)

filtered_data = bandpass_filter(data, lowcut, highcut, fs)

def wavelet_denoise(data, wavelet='db4', level=4):
    coeffs = pywt.wavedec(data, wavelet, level=level)
    sigma = np.median(np.abs(coeffs[-1])) / 0.6745  # Noise estimation
    threshold = sigma * np.sqrt(2 * np.log(len(data)))
    coeffs_denoised = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]

    # Reconstruct the signal
    denoised = pywt.waverec(coeffs_denoised, wavelet)

    # Ensure the output length matches the input
    if len(denoised) > len(data):
        denoised = denoised[:len(data)]
    elif len(denoised) < len(data):
        denoised = np.pad(denoised, (0, len(data) - len(denoised)), mode='edge')

    return denoised

denoised_data = wavelet_denoise(filtered_data)

# 3. STA/LTA Detection
def sta_lta(data, fs, sta_window=1, lta_window=10, threshold=3):
    sta_samples = int(sta_window * fs)
    lta_samples = int(lta_window * fs)

    sta = np.convolve(data**2, np.ones(sta_samples)/sta_samples, mode='same')
    lta = np.convolve(data**2, np.ones(lta_samples)/lta_samples, mode='same')

    lta[lta == 0] = np.finfo(float).eps  # Avoid division by zero
    ratio = sta / lta

    event_indices = np.where(ratio > threshold)[0]
    return event_indices, ratio

event_indices, sta_lta_ratio = sta_lta(denoised_data, fs)

# 4. Plot Results
plt.figure(figsize=(12, 6))
plt.subplot(3, 1, 1)
plt.plot(t, data, label="Raw Data", alpha=0.6)
plt.title("Raw Underwater Acoustic Signal")
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(t, filtered_data, label="Filtered & Denoised", color='g')
plt.title("Filtered & Denoised Signal")
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(t, sta_lta_ratio, label="STA/LTA Ratio", color='r')
plt.axhline(y=3, color='k', linestyle="--", label="Threshold")
plt.scatter(t[event_indices], sta_lta_ratio[event_indices], color='black', marker='o', label="Detected P-Wave")
plt.title("STA/LTA P-Wave Detection")
plt.legend()

plt.tight_layout()
plt.show()


Using matplotlib backend: qtagg


C:\Users\Romain\AppData\Local\Temp\ipykernel_3136\1867890520.py:53: RuntimeWarning: overflow encountered in square
  sta = np.convolve(data**2, np.ones(sta_samples)/sta_samples, mode='same')
C:\Users\Romain\AppData\Local\Temp\ipykernel_3136\1867890520.py:54: RuntimeWarning: overflow encountered in square
  lta = np.convolve(data**2, np.ones(lta_samples)/lta_samples, mode='same')
C:\Users\Romain\AppData\Local\Temp\ipykernel_3136\1867890520.py:57: RuntimeWarning: invalid value encountered in divide
  ratio = sta / lta


In [13]:
import numpy as np
import scipy.signal as signal
import pywt
import matplotlib.pyplot as plt

# Constants
fs = 240  # Sampling frequency (Hz)
# Adapted filter parameters for teleseismic P-waves
lowcut = 0.6   # Lower cutoff frequency (Hz)
highcut = 2.0  # Upper cutoff frequency (Hz)
t = times
data = data
# Assuming 'data' and 't' are already defined
n = len(data)  # Number of samples
p_wave_time = 0.5 * n / fs  # Expected P-wave time
data_norm = data / np.max(np.abs(data))


# 1. Band-Pass Filter
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data), b, a

filtered_data, b, a = bandpass_filter(data_norm, lowcut, highcut, fs)

# 2. Wavelet Denoising
def wavelet_denoise(data, wavelet='sym5', level=4):
    coeffs = pywt.wavedec(data, wavelet, level=level)
    sigma = np.median(np.abs(coeffs[-1])) / 0.6745  # Noise estimation
    threshold = sigma * np.sqrt(2 * np.log(len(data)))
    coeffs_denoised = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]

    denoised = pywt.waverec(coeffs_denoised, wavelet)

    if len(denoised) > len(data):
        denoised = denoised[:len(data)]
    elif len(denoised) < len(data):
        denoised = np.pad(denoised, (0, len(data) - len(denoised)), mode='edge')

    return denoised

denoised_data = wavelet_denoise(filtered_data)

# 3. STA/LTA Detection
def sta_lta(data, fs, sta_window=1, lta_window=10, threshold=3):
    sta_samples = int(sta_window * fs)
    lta_samples = int(lta_window * fs)

    sta = np.convolve(data**2, np.ones(sta_samples)/sta_samples, mode='same')
    lta = np.convolve(data**2, np.ones(lta_samples)/lta_samples, mode='same')

    lta[lta == 0] = np.finfo(float).eps  # Avoid division by zero
    ratio = sta / lta

    event_indices = np.where(ratio > threshold)[0]
    return event_indices, ratio

event_indices, sta_lta_ratio = sta_lta(denoised_data, fs)

# 4. Detect P-Wave in STA/LTA
detected_p_wave_time = event_indices[0] / fs if len(event_indices) > 0 else None

# 5. Plot Results
plt.figure(figsize=(12, 8))

# (A) Plot Raw Data
plt.subplot(4, 1, 1)
plt.plot(t, data, label="Raw Data", alpha=0.6)
plt.axvline(p_wave_time, color='r', linestyle="--", label="Expected P-Wave")
plt.title("Raw Underwater Acoustic Signal")
plt.legend()

# (B) Plot Filtered Data
plt.subplot(4, 1, 2)
plt.plot(t, filtered_data, label="Band-Pass Filtered (0.5–15 Hz)", color='g')
plt.axvline(p_wave_time, color='r', linestyle="--", label="Expected P-Wave")
plt.title("Band-Pass Filtered Signal")
plt.legend()

# (C) Plot Denoised Data
plt.subplot(4, 1, 3)
plt.plot(t, denoised_data, label="Wavelet Denoised", color='b')
plt.axvline(p_wave_time, color='r', linestyle="--", label="Expected P-Wave")
if detected_p_wave_time:
    plt.axvline(detected_p_wave_time, color='black', linestyle=":", label="Detected P-Wave")
plt.title("Filtered & Wavelet Denoised Signal")
plt.legend()

# (D) STA/LTA Detection
plt.subplot(4, 1, 4)
plt.plot(t, sta_lta_ratio, label="STA/LTA Ratio", color='r')
plt.axhline(y=3, color='k', linestyle="--", label="Threshold")
plt.axvline(p_wave_time, color='r', linestyle="--", label="Expected P-Wave")
if detected_p_wave_time:
    plt.axvline(detected_p_wave_time, color='black', linestyle=":", label="Detected P-Wave")
plt.title("STA/LTA P-Wave Detection")
plt.legend()

plt.tight_layout()
plt.show()

# 6. Plot Band-Pass Filter Response
w, h = signal.freqz(b, a, worN=1024, fs=fs)
plt.figure(figsize=(8, 4))
plt.plot(w, abs(h), 'b')
plt.title("Band-Pass Filter Frequency Response (0.5–15 Hz)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Gain")
plt.grid()
plt.show()

# Print Detected P-Wave Time
if detected_p_wave_time:
    print(f"Detected P-wave at {detected_p_wave_time:.2f} seconds (Expected ~{p_wave_time:.2f}s)")
else:
    print(f"No P-wave detected. Expected at {p_wave_time:.2f} seconds.")


No P-wave detected. Expected at 299.99 seconds.


In [14]:
%matplotlib qt
plt.figure(figsize=(12, 8))
plt.plot(t,filtered_data, color='r')
plt.plot(t, denoised_data, label="Wavelet Denoised", color='b')
plt.axvline(p_wave_time, color='r', linestyle="--", label="Expected P-Wave")

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, detrend, spectrogram

# Load data

fs = 240
time = np.arange(len(data)) / fs

# Detrend
data = detrend(data, type='linear')

# Design filters (MATLAB parameters)
nyquist = fs / 2
b_high, a_high = butter(4, 0.06 , btype='high', fs=240)  # High-pass
b_low, a_low = butter(6, 2.1 , btype='low', fs=240)     # Low-pass

# 3. Apply filters with MATLAB-like padding
filtered_high = filtfilt(b_high, a_high, data, padtype='constant', padlen=30)
filtered_data = filtfilt(b_low, a_low, filtered_high, padtype='constant', padlen=30)

# 4. Plot results
plt.plot(time, filtered_data, 'b', linewidth=0.75)
plt.title("Python Output (MATLAB-like Padding)")
plt.show()

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import filtfilt

# Sampling frequency and time vector
Fs = 250                # 100 Hz sampling frequency
N = n                 # Number of samples (adjust as needed)
time =times

# Simulated seismic signal:
# For example, a low-frequency sine with some noise and a higher-frequency component.
signal =data
# Define filter coefficients exactly as in MATLAB for Fs = 100 Hz:
#
# Filter 1: Hippocampe OBS (cut ~1 Hz)
# MATLAB: a = [1, -94/100, 0], b = [1, -1, 0]
b1 = np.array([1, -1, 0])
a1 = np.array([1, -94/240, 0])

# Filter 2: Mermaid electronic card (cut ~0.1 Hz; weak filter)
# MATLAB: a = [1, -9965/10000, 0], b = [1, -1, 0]
b2 = np.array([1, -1, 0])
a2 = np.array([1, -9965/240, 0])

# Filter 3: High-pass filter cutting approximately at 0.3 Hz
# MATLAB: a = [1, -99/100, 0], b = [1, -1, 0]
b3 = np.array([1, -1, 0])
a3 = np.array([1, -99/240, 0])

# Apply zero-phase filtering (filtfilt applies forward and backward filtering)
filtered_signal1 = filtfilt(b1, a1, signal)
filtered_signal2 = filtfilt(b2, a2, signal)
filtered_signal3 = filtfilt(b3, a3, signal)

# Plotting the original and filtered signals
plt.figure(figsize=(12, 8))
plt.plot(time, signal, label='Original Signal', color='gray', alpha=0.5)
plt.plot(time, filtered_signal1, label='Hippocampe OBS (cut ~1 Hz)', linewidth=2)
plt.plot(time, filtered_signal2, label='Mermaid Electronic Card (cut ~0.1 Hz)', linewidth=2, linestyle='--')
plt.plot(time, filtered_signal3, label='HP Filter (cut ~0.3 Hz)', linewidth=2, linestyle='-.')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Seismic Signal Filtering with MATLAB-like Coefficients')
plt.legend()
plt.tight_layout()
plt.show()


In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import os
from matplotlib.colors import LinearSegmentedColormap



# Function to downsample audio data
def downsample_audio(data, original_fs, target_fs):
    """Downsample audio data to the target frequency"""
    print(f"Downsampling from {original_fs}Hz to {target_fs}Hz")
    # Calculate downsampling factor
    factor = int(original_fs / target_fs)
    # Apply anti-aliasing filter before downsampling
    b, a = signal.butter(5, target_fs/2, fs=original_fs, btype='low')
    filtered_data = signal.filtfilt(b, a, data)
    # Downsample by taking every 'factor' sample
    downsampled_data = filtered_data[::factor]
    return downsampled_data, target_fs

# Function to apply dehazing (using spectral subtraction)
def dehaze_audio(data, fs, frame_size=1024, overlap=0.8):
    """Apply spectral subtraction for dehazing"""
    print("Applying dehazing using spectral subtraction")
    hop_size = int(frame_size * (1 - overlap))
    # Estimate noise profile from first few frames
    num_noise_frames = 5
    noise_estimate = np.zeros(frame_size // 2 + 1)

    frames = []
    for i in range(0, len(data) - frame_size, hop_size):
        frame = data[i:i+frame_size]
        if len(frame) < frame_size:
            frame = np.pad(frame, (0, frame_size - len(frame)))
        frames.append(frame)

    # Estimate noise from first few frames
    for i in range(min(num_noise_frames, len(frames))):
        noise_frame = frames[i]
        noise_spectrum = np.abs(np.fft.rfft(noise_frame * np.hanning(frame_size)))
        noise_estimate += noise_spectrum / num_noise_frames

    # Apply spectral subtraction
    result = np.zeros(len(data))
    window = np.hanning(frame_size)

    for i, frame in enumerate(frames):
        windowed_frame = frame * window
        spectrum = np.fft.rfft(windowed_frame)
        magnitude = np.abs(spectrum)
        phase = np.angle(spectrum)

        # Subtract noise and ensure no negative values
        magnitude = np.maximum(magnitude - noise_estimate * 1.5, 0.01 * magnitude)

        # Reconstruct frame
        enhanced_spectrum = magnitude * np.exp(1j * phase)
        enhanced_frame = np.fft.irfft(enhanced_spectrum)

        # Overlap-add
        start = i * hop_size
        end = start + frame_size
        result[start:end] += enhanced_frame

    # Normalize
    result = result / np.max(np.abs(result))
    return result

# Function to apply Butterworth bandpass filter
def apply_butter_bandpass(data, fs, lowcut, highcut, order=5):
    """Apply Butterworth bandpass filter"""
    print(f"Applying bandpass filter: {lowcut}-{highcut}Hz, order {order}")
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    filtered_data = signal.filtfilt(b, a, data)
    return filtered_data

# Function to create spectrogram
def create_spectrogram(data, fs, nperseg=256, noverlap=128, cmap='viridis'):
    """Create and return spectrogram of the data"""
    print("Creating spectrogram")
    f, t, Sxx = signal.spectrogram(data, fs=fs, nperseg=nperseg, noverlap=noverlap)
    return f, t, Sxx

# Function to detect potential seismic events using energy
def detect_seismic_events(data, fs, window_size=5.0, threshold_factor=20.0):
    """Detect potential seismic events based on energy threshold"""
    print("Detecting potential seismic events")
    window_samples = int(window_size * fs)
    energy = []

    # Calculate energy in sliding windows
    for i in range(0, len(data) - window_samples, window_samples // 2):
        window = data[i:i+window_samples]
        window_energy = np.sum(window**2) / len(window)
        energy.append(window_energy)

    # Set threshold as a factor of the median energy
    energy = np.array(energy)
    threshold = np.median(energy) * threshold_factor

    # Find events that exceed threshold
    events = []
    in_event = False
    event_start = 0

    for i, e in enumerate(energy):
        if e > threshold and not in_event:
            in_event = True
            event_start = i * (window_samples // 2) / fs
        elif e <= threshold and in_event:
            in_event = False
            event_end = i * (window_samples // 2) / fs
            events.append((event_start, event_end))

    # Handle if we're still in an event at the end
    if in_event:
        event_end = len(data) / fs
        events.append((event_start, event_end))

    return events, energy, threshold

# Main processing function
def process_underwater_recording(data,original_fs, target_fs=50, low_pass=1.5, high_pass=0.6):
    """Process underwater recording to visualize seismic events"""
    # Load data

    print(f"Original sampling rate: {original_fs}Hz")
    print(f"Original data length: {len(data)} samples ({len(data)/original_fs:.2f} seconds)")

    # Downsample to 50Hz
    downsampled_data, new_fs = downsample_audio(data, original_fs, target_fs)
    print(f"Downsampled data length: {len(downsampled_data)} samples ({len(downsampled_data)/new_fs:.2f} seconds)")

    # Apply dehazing
    dehazed_data = dehaze_audio(downsampled_data, new_fs)

    # Apply Butterworth bandpass filter
    filtered_data = apply_butter_bandpass(dehazed_data, new_fs, high_pass, low_pass)

    # Create spectrograms for comparison
    f_orig, t_orig, Sxx_orig = create_spectrogram(downsampled_data, new_fs)
    f_filtered, t_filtered, Sxx_filtered = create_spectrogram(filtered_data, new_fs)

    # Detect potential seismic events
    events, energy, threshold = detect_seismic_events(filtered_data, new_fs)

    # Create custom colormap for better visualization (dark blue to light blue to white)
    colors = [(0, 0, 0.5), (0, 0, 1), (0, 0.5, 1), (0, 1, 1), (1, 1, 1)]
    positions = [0, 0.25, 0.5, 0.75, 1]
    cmap_name = 'seismic_blue'
    cm = LinearSegmentedColormap.from_list(cmap_name, list(zip(positions, colors)))

    # Visualize the results
    plt.figure(figsize=(15, 10))

    # Plot original waveform
    plt.subplot(4, 1, 1)
    plt.plot(np.arange(len(downsampled_data))/new_fs, downsampled_data)
    plt.title('Original Signal (Downsampled to 50Hz)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    # Plot filtered waveform with event markers
    plt.subplot(4, 1, 2)
    plt.plot(np.arange(len(filtered_data))/new_fs, filtered_data)
    plt.title(f'Filtered Signal (Bandpass {high_pass}-{low_pass}Hz)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    # Add markers for detected events
    for event_start, event_end in events:
        plt.axvspan(event_start, event_end, color='red', alpha=0.3)
        plt.annotate('Event', xy=(event_start, max(filtered_data)*0.8),
                    xytext=(event_start, max(filtered_data)),
                    arrowprops=dict(facecolor='red', shrink=0.05))

    # Plot original spectrogram
    plt.subplot(4, 1, 3)
    plt.pcolormesh(t_orig, f_orig, 10 * np.log10(Sxx_orig + 1e-10), shading='gouraud', cmap=cm)
    plt.title('Original Spectrogram')
    plt.ylabel('Frequency (Hz)')
    plt.xlabel('Time (s)')
    plt.colorbar(label='Power/Frequency (dB/Hz)')
    plt.ylim(0, 10)  # Limit frequency range for better visualization

    # Plot filtered spectrogram
    plt.subplot(4, 1, 4)
    plt.pcolormesh(t_filtered, f_filtered, 10 * np.log10(Sxx_filtered + 1e-10), shading='gouraud', cmap=cm)
    plt.title('Filtered Spectrogram')
    plt.ylabel('Frequency (Hz)')
    plt.xlabel('Time (s)')
    plt.colorbar(label='Power/Frequency (dB/Hz)')
    plt.ylim(0, 5)  # Limit frequency range for better visualization

    # Add markers for detected events on spectrogram
    for event_start, event_end in events:
        plt.axvspan(event_start, event_end, color='red', alpha=0.3)

    plt.tight_layout()

    # Create a separate figure for event detection results
    plt.figure(figsize=(10, 4))
    window_size = 1.0  # seconds
    time_axis = np.arange(len(energy)) * (window_size/2)
    plt.plot(time_axis, energy)
    plt.axhline(threshold, color='r', linestyle='--', label='Threshold')
    plt.title('Signal Energy for Event Detection')
    plt.xlabel('Time (s)')
    plt.ylabel('Energy')
    plt.legend()

    # Print detected events
    print("/nDetected seismic events:")
    for i, (start, end) in enumerate(events):
        print(f"Event {i+1}: {start:.2f}s - {end:.2f}s (duration: {end-start:.2f}s)")

    # Save processed data if needed
    # wavfile.write("filtered_underwater_recording.wav", new_fs, (filtered_data * 32767).astype(np.int16))

    plt.show()
    return filtered_data, events

# Example usage
if __name__ == "__main__":
    # Replace with your actual file path
    file_path = "underwater_recording.wav"

    # Process with the specific parameters
    # Original frequency: 240Hz
    # Target frequency: 50Hz
    # Filter range: 0.6-1.5Hz
    filtered_data, events = process_underwater_recording(
        data,240,
        target_fs=60,
        low_pass=2.5,
        high_pass=0.6
    )

Original sampling rate: 240Hz
Original data length: 143997 samples (599.99 seconds)
Downsampling from 240Hz to 60Hz
Downsampled data length: 36000 samples (600.00 seconds)
Applying dehazing using spectral subtraction
Applying bandpass filter: 0.6-2.5Hz, order 5
Creating spectrogram
Creating spectrogram
Detecting potential seismic events
/nDetected seismic events:
Event 1: 300.00s - 317.50s (duration: 17.50s)
Event 2: 320.00s - 357.50s (duration: 37.50s)
Event 3: 365.00s - 377.50s (duration: 12.50s)


In [18]:
# Downsample to 50Hz
target_fs=120
low_pass=2
high_pass=0.06
original_fs = 240

downsampled_data, new_fs = downsample_audio(data, original_fs, target_fs)
print(f"Downsampled data length: {len(downsampled_data)} samples ({len(downsampled_data)/new_fs:.2f} seconds)")

# Apply dehazing
dehazed_data = dehaze_audio(downsampled_data, target_fs)

# Apply Butterworth bandpass filter
filtered_data = apply_butter_bandpass(dehazed_data, target_fs, high_pass, low_pass)

Downsampling from 240Hz to 120Hz
Downsampled data length: 71999 samples (599.99 seconds)
Applying dehazing using spectral subtraction
Applying bandpass filter: 0.06-2Hz, order 5


In [19]:
%matplotlib qt
# plt.plot(data,label='Original Signal')
# plt.plot(downsampled_data,label='Downsampled Signal')
t = np.arange(len(filtered_data))/30

plt.plot(t,filtered_data,label='Filtered Signal')
print(filtered_data)


[nan nan nan ... nan nan nan]


In [20]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'audio_data' is your numpy array and fs = 240 Hz
audio_segment = data # Take a segment for FFT (e.g., 10 seconds)

fft_result = np.fft.fft(audio_segment)
frequencies = np.fft.fftfreq(audio_segment.size, d=1/fs)
magnitude_spectrum = np.abs(fft_result)

plt.figure(figsize=(10, 5))
plt.plot(frequencies[:audio_segment.size//2], magnitude_spectrum[:audio_segment.size//2]) # Plot positive frequencies
plt.title('Frequency Spectrum of Audio Data (Example Segment)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.xlim(0, fs/2) # Limit x-axis to Nyquist frequency
plt.grid(True)
plt.show()

In [21]:
import numpy as np
from scipy.signal import butter, sosfiltfilt
import matplotlib.pyplot as plt

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    """
    Applies a bandpass filter to the audio data.

    Args:
        data (np.ndarray): Audio data.
        lowcut (float): Lower cutoff frequency in Hz.
        highcut (float): Upper cutoff frequency in Hz.
        fs (float): Sampling rate of the audio data in Hz.
        order (int): Order of the filter. Higher order filters have steeper
                     roll-offs but may introduce more ringing.

    Returns:
        np.ndarray: Filtered audio data.
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    sos = butter(order, [low, high], btype='band', output='sos')
    filtered_data = sosfiltfilt(sos, data)
    return filtered_data

# Example usage:
# Assuming 'audio_data' is your numpy array of audio data and 'sampling_rate' is the sampling rate.
# You'll need to define appropriate 'lowcut' and 'highcut' frequencies based on your P-wave characteristics and noise.
# For example, let's assume P-waves are roughly in the range of 5-20 Hz (this is just an example, adjust accordingly!).
sampling_rate = 240.0  # Example sampling rate, adjust to your actual rate
lowcut_freq =5    # Lower cutoff frequency for bandpass filter
highcut_freq =30 # Upper cutoff frequency for bandpass filter

# Let's create some dummy audio data for demonstration
duration = 2*60  # seconds
time = np.arange(len(data))/sampling_rate
# A signal with frequencies around 10Hz, plus some noise
audio_data = data

filtered_audio = bandpass_filter(audio_data, lowcut_freq, highcut_freq, sampling_rate)

# Plotting original and filtered data for comparison
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(time, audio_data)
plt.title('Original Audio Data')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.subplot(2, 1, 2)
plt.plot(time, filtered_audio)
plt.title(f'Bandpass Filtered Audio ({lowcut_freq}-{highcut_freq} Hz)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()

In [22]:
from scipy.signal import wiener

def deconvolve_wiener(data, noise_power=0.000001): # noise_power is a parameter to tune
    """
    Applies Wiener deconvolution to the audio data.

    Args:
        data (np.ndarray): Audio data to be deconvolved.
        noise_power (float): Estimated noise power relative to signal power.
                             Adjust this based on your data's noise characteristics.

    Returns:
        np.ndarray: Deconvolved audio data.
    """
    deconvolved_data = wiener(data, noise=noise_power)
    return deconvolved_data

# Example usage:
# Apply deconvolution after filtering (or on original data, try both)
deconvolved_audio = deconvolve_wiener(filtered_audio, noise_power=0.01) # Adjust noise_power

# Plotting filtered and deconvolved data for comparison
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(time, filtered_audio)
plt.title('Bandpass Filtered Audio')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.subplot(2, 1, 2)
plt.plot(time, deconvolved_audio)
plt.title('Deconvolved Audio (Wiener)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()

In [23]:
def normalize_amplitude(data):
    max_amplitude = np.max(np.abs(data))
    if max_amplitude > 0:
        return data / max_amplitude
    else:
        return data # Avoid division by zero if data is all zeros

normalized_audio = normalize_amplitude(deconvolved_audio) # Or stacked_audio, etc.
plt.plot(time, normalized_audio)

In [24]:
import numpy as np
from scipy.signal import filtfilt, freqz
import matplotlib.pyplot as plt

# High-pass filter coefficients (4th order Butterworth)
a1 = np.array([1.0000, -3.9959, 5.9877, -3.9877, 0.9959], dtype=np.float64)
b1 = np.array([0.9979, -3.9918, 5.9877, -3.9918, 0.9979], dtype=np.float64)

# Low-pass filter coefficients (6th order Butterworth)
a2 = np.array([1.0000, -5.7876, 13.9604, -17.9641, 13.0061, -5.0234, 0.8086], dtype=np.float64)
b2 = np.array([3.8857e-10, 2.3314e-09, 5.8286e-09, 7.7714e-09,
              5.8286e-09, 2.3314e-09, 3.8857e-10], dtype=np.float64)

def apply_matlab_filters(data, fs):
    """Replicates MATLAB's filtering exactly using provided coefficients."""
    # MATLAB-style padding for filtfilt
    filtered = filtfilt(b1, a1, data, padtype='constant', padlen=30)  # High-pass
    filtered = filtfilt(b2, a2, filtered, padtype='constant', padlen=30)  # Low-pass
    return filtered

def plot_response(b, a, fs, title):
    w, h = freqz(b, a, worN=2000, fs=fs)
    plt.plot(w, 20 * np.log10(abs(h)))
    plt.title(title)
    plt.ylabel('Amplitude [dB]')
    plt.xlabel('Frequency [Hz]')

plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plot_response(b1, a1, fs=240, title='High-pass Filter Response (0.6Hz cutoff)')
plt.subplot(2, 1, 2)
plot_response(b2, a2, fs=240, title='Low-pass Filter Response (2.1Hz cutoff)')
plt.tight_layout()
plt.show()

In [25]:
def process_signal(data):
    # Load data (replace with your data loading)
    # Must use float64 for precision

    # MATLAB-style processing
    processed = apply_matlab_filters(data, fs=240)
    print(processed)
    # Plotting comparison
    plt.figure(figsize=(15, 5))
    plt.plot(data, 'b', alpha=0.5, label='Original')
    plt.plot(processed, 'r', label='Filtered (MATLAB coefficients)')
    plt.legend()
    plt.title('MATLAB vs Python Filtering Comparison')
    plt.show()

    return processed

# Example usage:
processed_data = process_signal(data)

[nan nan nan ... nan nan nan]


In [26]:
import datetime as dt
cycle = 4012802424
sple = 4004161980
fstsp = 8640447
start_date = dt.datetime.strptime("2018 01-16 07:30:50",'%Y %m-%d %H:%M:%S')
(sple/240.)


16684008.25

In [27]:
plt.close("all")

# **Estimate clock drift**


In [45]:
import pandas as pd
from datetime import timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt
name = 'NEAMS'
File = f'C:/Users/Romain/PycharmProjects/toolbox/src/utils/detection/seismic_arrival_picking_tool/ui/{name}_2018_pointed.csv'

data = pd.read_csv(File,parse_dates=['time', 'arrival_time', 'observed_arrival_time'], infer_datetime_format=True)


# data.drop('nst',axis=1,inplace=True)
data.dropna(how='any', inplace =True, ignore_index=True)
data['time'] = pd.to_datetime(data['time'], utc=True, errors='coerce')
ELAN_QC = [0.5,0.5,0.2,0.3,1,2,0.5,0.2,4,0.2,1,0.5,0.5,5,8,4,0.5,0.5,0.3,0.8,0.5,np.nan,1,0.5,0.2,0.3]
WKER_QC = [0.2,2,0.2,1,0.5,0.1,0.5,0.3]
NEAMS_QC = [0.5,0.5,0.1,1,0.3,0.4,0.2,0.3,1,0.4,0.2,1,0.4,0.2,0.2,0.1,0.3,0.4,0.5,0.5,0.1,0.5,0.2,1,0.5,0.5,0.2,0.2,0.5,0.5,0.5,0.5,0.8 ]
error = {"ELAN": ELAN_QC, "WKER2": WKER_QC, "NEAMS": NEAMS_QC}
sig = np.sqrt(np.power(2.5,2)+np.power(error[name],2))
data["arrival_uncertainty"] = pd.Series(sig, name="arrival_uncertainty")
data.dropna(how='any', inplace =True, ignore_index=True)
# data['observed_arrival_time'] -= timedelta(minutes=10)
# data.to_csv(File)

C:\Users\Romain\AppData\Local\Temp\ipykernel_3136\2539290049.py:10: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data = pd.read_csv(File,parse_dates=['time', 'arrival_time', 'observed_arrival_time'], infer_datetime_format=True)


In [47]:
data["time_diff"] = (data['arrival_time'] - data['observed_arrival_time'])
data["time_diff"] = data["time_diff"].dt.total_seconds()
data["arrival_uncertainty"] = 2.

In [48]:
travel_time_ELAN = pd.read_csv(f"C:/Users/Romain/PycharmProjects/Sea_sound_toolbox/data/2018_{name}_travel_times.csv")
#adding travel time from bottom to hydrophone
#C:/Users/Romain/PycharmProjects/Sea_sound_toolbox/data/2018_NEAMS_travel_times.csv
data['month'] = data['time'].dt.month
data['arrival_time'] += pd.to_timedelta(
    data['month'].map(travel_time_ELAN.set_index('month')['travel_time']),
    unit='s'
)

In [50]:
data

,Unnamed: 0,time,latitude,longitude,depth,mag,magType,gap,dmin,rms,...,phase,travel_time,arrival_time,distance_deg,candidate,predicted_arrival_time,observed_arrival_time,arrival_uncertainty,time_diff,month
0,0,2018-02-09 11:43:56.760000+00:00,-17.8709,-178.6605,556.91,6.0,mww,35.0,7.704,0.90,...,P,708.063557,2018-02-09 11:55:46.041848611+00:00,87.336078,True,2018-02-09 12:05:44.823556611+00:00,2018-02-09 11:55:44.825634336+00:00,2.0,-0.002078,2
1,1,2018-03-08 17:39:51.100000+00:00,-4.3762,153.1996,22.86,6.8,mww,16.0,1.049,1.05,...,P,673.772514,2018-03-08 17:51:06.090807269+00:00,70.665420,True,2018-03-08 18:01:04.872514269+00:00,2018-03-08 17:51:06.877507200+00:00,2.0,-2.004993,3
2,2,2018-03-24 11:23:32.050000+00:00,-5.4959,151.4971,33.00,6.3,mww,19.0,5.361,0.95,...,P,659.794798,2018-03-24 11:34:33.063091201+00:00,68.633734,True,2018-03-24 11:44:31.844798201+00:00,2018-03-24 11:34:33.849189312+00:00,2.0,-2.004391,3
3,3,2018-03-25 20:14:47.690000+00:00,-6.6343,129.8172,169.00,6.4,mww,13.0,1.988,1.09,...,P,517.305760,2018-03-25 20:23:26.214052678+00:00,50.043337,True,2018-03-25 20:33:24.995759678+00:00,2018-03-25 20:23:27.106314536+00:00,2.0,-2.110555,3
4,4,2018-03-26 09:51:00.430000+00:00,-5.5024,151.4025,40.00,6.7,mww,15.0,1.507,1.18,...,P,658.433871,2018-03-26 10:02:00.082164144+00:00,68.550296,True,2018-03-26 10:11:58.863871144+00:00,2018-03-26 10:02:02.871238130+00:00,2.0,-4.007367,3
5,5,2018-03-29 21:25:36.790000+00:00,-5.5321,151.4999,35.00,6.9,mww,18.0,1.487,1.02,...,P,659.404282,2018-03-29 21:36:37.412574939+00:00,68.617020,True,2018-03-29 21:46:36.194281939+00:00,2018-03-29 21:36:37.102910575+00:00,2.0,-0.908629,3
6,6,2018-04-02 05:57:35.680000+00:00,-24.7190,-176.8865,92.00,6.1,mww,20.0,4.603,1.15,...,P,745.378921,2018-04-02 06:10:02.277218316+00:00,85.049565,True,2018-04-02 06:20:01.058921316+00:00,2018-04-02 06:10:04.262463822+00:00,2.0,-3.203543,4
7,7,2018-04-02 13:40:34.840000+00:00,-20.6588,-63.0058,559.00,6.8,mww,16.0,5.320,0.99,...,PKIKP,1064.575853,2018-04-02 13:58:20.634149942+00:00,118.559320,True,2018-04-02 14:08:19.415852942+00:00,2018-04-02 13:58:24.474173382+00:00,2.0,-5.058320,4
8,8,2018-05-09 10:41:45.910000+00:00,36.9942,71.3822,116.00,6.2,mww,17.0,0.578,1.10,...,P,655.659107,2018-05-09 10:52:42.787420909+00:00,69.461404,True,2018-05-09 11:02:41.569106909+00:00,2018-05-09 10:52:40.603109442+00:00,2.0,0.965997,5
9,9,2018-05-18 01:45:31.460000+00:00,-34.5886,-178.4143,11.00,6.1,mww,19.0,3.982,0.82,...,P,722.397664,2018-05-18 01:57:35.075977673+00:00,78.727242,True,2018-05-18 02:07:33.857663673+00:00,2018-05-18 01:57:38.507276787+00:00,2.0,-4.649613,5


In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt
# Assuming times_df exists and contains:
#  'expected_arrival' (datetime stored as nanoseconds),
#  'pointed_arrival_time' (datetime stored as nanoseconds),
#  'arrival_uncertainty' (uncertainty in nanoseconds)

df = data.copy()

# Define a list of file names to exclude
exclude_files = []#['00000794.DAT','00000796.DAT']

# Create a mask to keep only the rows that are NOT in the exclude list
# mask = ~df['name'].isin(exclude_files)
# df = df[mask]
# print("DataFrame:")
# print(df)

# Convert times to seconds (as float)
x_abs = df['arrival_time'].values.astype(np.float64) / 1e9  # absolute expected times in seconds
y_abs = df['observed_arrival_time'].values.astype(np.float64) / 1e9 # absolute measured times in seconds
e_sec = df['arrival_uncertainty'].values.astype(np.float64)  # uncertainty in seconds

# Use weights = 1/uncertainty for regression on absolute times
weights = 1 / e_sec

# Main weighted regression on absolute times:
# Model: y_abs = a*x_abs + b
(a, b), cov = np.polyfit(x_abs, y_abs, 1, w=weights, cov=True)
var_a, var_b = np.diag(cov)
sigma_a = np.sqrt(var_a)
sigma_b = np.sqrt(var_b)

ci_a = 1.96 * sigma_a
ci_b = 1.96 * sigma_b

drift_ppm = (a - 1) * 1e6
ci_a_ppm = ci_a * 1e6

print("\nWeighted Least Squares Regression Results (with 95% CI):")
print(f"Drift factor (a): {a:.12f}")
print(f"Offset (b): {b:.3f} s")
print(f"Drift (a - 1) in ppm: {drift_ppm:.3f} ppm (95% CI) :[ {drift_ppm - ci_a_ppm:.3f}: {drift_ppm + ci_a_ppm:.3f}] ")



# --- Make a more readable scale for plotting ---

# Use relative time: subtract the first expected arrival time.
x0 = x_abs.min()  # reference time (first expected arrival)
x_rel = x_abs - x0  # in seconds; now the x-axis starts at 0

# Compute the drift (difference) in seconds and convert to microseconds for readability.
drift_sec = y_abs - x_abs   # drift in seconds
drift_us = drift_sec * 1e6   # convert to microseconds
# Similarly, uncertainty in drift is the same as e_sec (if only measured arrival has uncertainty)
e_drift_us = e_sec * 1e6     # uncertainty in microseconds

# Optional: Recompute regression for the drift vs. time.
# Note that: y_abs = a*x_abs + b  =>  drift = y_abs - x_abs = (a-1)*x_abs + b.
# But if you shift x_abs -> x_rel = x_abs - x0, then:
# drift = (a-1)*(x_rel+x0) + b = (a-1)*x_rel + [b + (a-1)*x0].
drift_slope_us = (a - 1) * 1e6  # in μs per second (same as ppm, since 1 ppm = 1e-6)
drift_intercept_us = (b + (a - 1) * x0) * 1e6

# For plotting, let's convert the relative time axis to hours
x_hours = x_rel / 3600.0  # relative time in hours

# Generate fitted drift values using the regression on absolute times.
# We'll compute drift (in μs) as: drift_fit_us = (a-1)*x_abs + b, then convert to μs.
drift_fit_us = ((a - 1) * (x_rel + x0) + b) * 1e6

# Plot the drift (in μs) versus relative time (in hours) with error bars.
plt.figure()
plt.errorbar(x_hours, drift_us, yerr=e_drift_us, fmt='o', capsize=5, label='Data (μs drift)')
plt.plot(x_hours, drift_fit_us, 'r-',
         label=f'Regression: slope = {drift_slope_us:.3f} μs/s, '
               f'ppm (95% CI) :[{drift_ppm - ci_a_ppm:.3f}:{drift_ppm + ci_a_ppm:.3f}]')
plt.xlabel('Time from first expected arrival (hours)')
plt.ylabel('Drift (measured - expected) in μs')
plt.title("Time Drift (Measured vs Expected) with Uncertainty")
# for i, name in enumerate(df['name']):
#     name = name.split('.')[0][-4::]
#     plt.annotate(name, (x_hours[i], drift_us[i]),
#                  textcoords="offset points", xytext=(-5,5), ha="right", fontsize=8)
plt.legend()
plt.show()






Weighted Least Squares Regression Results (with 95% CI):
Drift factor (a): 1.000000140071
Offset (b): -212.290 s
Drift (a - 1) in ppm: 0.140 ppm (95% CI) :[ 0.074: 0.206] 


In [52]:
df = data.copy()
import datetime as dt
# Convert times to seconds (as float)
x_abs = df['arrival_time'].values.astype(np.float64) / 1e9
y_abs = df['observed_arrival_time'].values.astype(np.float64) / 1e9 # remove 10-minute addition
e_sec = df['arrival_uncertainty'].values.astype(np.float64) # uncertainty in seconds

# Compute drift (observed - expected) in seconds
drift = y_abs - x_abs

# Create a relative time axis by subtracting the minimum expected arrival time
x0 = dt.datetime.strptime('2018-01-16 17:34:34','%Y-%m-%d %H:%M:%S')
x0 = x0.timestamp() # reference time (first expected arrival)
x_rel = x_abs - x0  # now x_rel starts at 0

# Use weights = 1/uncertainty for regression on drift
weights = 1 / e_sec

# Perform weighted regression: drift = slope * x_rel + offset
(slope, offset), cov = np.polyfit(x_rel, drift, 1, w=weights, cov=True)
var_slope, var_offset = np.diag(cov)
sigma_slope = np.sqrt(var_slope)
sigma_offset = np.sqrt(var_offset)

# For clarity, express the drift slope in parts per million (ppm) and offset in microseconds
drift_ppm = slope * 1e6
ci_slope_ppm = 1.96 * sigma_slope * 1e6  # 95% CI for slope in ppm
offset_us = offset * 1e6  # offset at x_rel=0 in microseconds

print("\nWeighted Least Squares Regression Results (drift vs. relative time):")
print(f"Drift slope: {slope:.12f} s/s, i.e. {drift_ppm:.3f} ppm (95% CI: [{drift_ppm - ci_slope_ppm:.3f}:{drift_ppm + ci_slope_ppm:.3f}])")
print(f"Drift offset at reference time: {offset:.3f} s, i.e. {offset_us:.3f} μs")

# For plotting, compute the fitted drift values and convert to microseconds
drift_fit = slope * x_rel + offset
drift_fit_us = drift_fit * 1e6
drift_us = drift * 1e6  # convert measured drift to microseconds
e_drift_us = e_sec * 1e6  # uncertainty in microseconds

# Convert relative time to hours for plotting
x_hours = x_rel / 3600.0

plt.figure()
plt.errorbar(x_hours, drift_us, yerr=e_drift_us, fmt='o', capsize=5, label='Data (μs drift)')
plt.plot(x_hours, drift_fit_us, 'r-',
         label=f'Regression: slope = {drift_ppm:.3f} ppm (95% CI: [{drift_ppm - ci_slope_ppm:.3f}:{drift_ppm + ci_slope_ppm:.3f}])')
plt.xlabel('Time from first expected arrival (hours)')
plt.ylabel('Drift (observed - expected) in μs')
plt.title("Time Drift (Measured vs. Expected) with Uncertainty")
for i, name in enumerate(df['place']):
    short_name = name.split('.')[0][-4:]  # extract a shorter name
    plt.annotate(short_name, (x_hours[i], drift_us[i]),
                 textcoords="offset points", xytext=(-5, 5), ha="right", fontsize=8)
plt.legend()
plt.show()



Weighted Least Squares Regression Results (drift vs. relative time):
Drift slope: 0.000000140071 s/s, i.e. 0.140 ppm (95% CI: [0.074:0.206])
Drift offset at reference time: 0.076 s, i.e. 76094.474 μs


In [36]:
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from scipy import stats

def enhanced_drift_analysis(df):
    """
    Perform enhanced drift analysis with weighted least squares regression
    and comprehensive residual diagnostics.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing drift analysis data

    Returns:
    --------
    dict : Dictionary with regression results and diagnostic information
    """
    # Convert times to seconds (as float)
    x_abs = df['arrival_time'].values.astype(np.float64) / 1e9
    y_abs = df['observed_arrival_time'].values.astype(np.float64) / 1e9  # remove 10-minute addition
    e_sec = df['arrival_uncertainty'].values.astype(np.float64)  # uncertainty in seconds

    # Compute drift (observed - expected) in seconds
    drift = y_abs - x_abs

    # Create a relative time axis by subtracting the minimum expected arrival time
    x0 = dt.datetime.strptime('2018-01-16 17:34:34','%Y-%m-%d %H:%M:%S')
    x0 = x0.timestamp()  # reference time (first expected arrival)
    x_rel = x_abs - x0  # now x_rel starts at 0

    # Use weights = 1/uncertainty for regression on drift
    weights = 1 / e_sec

    # Perform weighted regression: drift = slope * x_rel + offset
    (slope, offset), cov = np.polyfit(x_rel, drift, 1, w=weights, cov=True)
    var_slope, var_offset = np.diag(cov)
    sigma_slope = np.sqrt(var_slope)
    sigma_offset = np.sqrt(var_offset)

    # Compute drift in parts per million (ppm)
    drift_ppm = slope * 1e6
    ci_slope_ppm = 1.96 * sigma_slope * 1e6  # 95% CI for slope in ppm
    offset_us = offset * 1e6  # offset at x_rel=0 in microseconds

    # Compute fitted drift and residuals
    drift_fit = slope * x_rel + offset
    residuals = drift - drift_fit

    # Residual diagnostics
    # Compute standardized residuals
    std_residuals = residuals / np.sqrt(e_sec**2 + (sigma_slope * x_rel)**2)

    # Statistical test on residuals
    _, shapiro_p = stats.shapiro(std_residuals)

    # Plotting
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10),
                                   gridspec_kw={'height_ratios': [3, 1]})

    # Top plot: Drift Data with Regression Line
    x_hours = x_rel / 3600.0
    drift_us = drift * 1e6  # convert measured drift to microseconds
    drift_fit_us = drift_fit * 1e6
    e_drift_us = e_sec * 1e6  # uncertainty in microseconds

    ax1.errorbar(x_hours, drift_us, yerr=e_drift_us, fmt='o', capsize=5,
                 label='Data (μs drift)')
    ax1.plot(x_hours, drift_fit_us, 'r-',
             label=f'Regression: slope = {drift_ppm:.3f} ppm\n95% CI: [{drift_ppm - ci_slope_ppm:.3f}:{drift_ppm + ci_slope_ppm:.3f}]')

    for i, name in enumerate(df['place']):
        short_name = name.split('.')[0][-4:]  # extract a shorter name
        ax1.annotate(short_name, (x_hours[i], drift_us[i]),
                     textcoords="offset points", xytext=(-5, 5),
                     ha="right", fontsize=8)

    ax1.set_xlabel('Time from first expected arrival (hours)')
    ax1.set_ylabel('Drift (observed - expected) in μs')
    ax1.set_title("Time Drift (Measured vs. Expected) with Uncertainty")
    ax1.legend()

    # Bottom plot: Residuals
    ax2.scatter(x_hours, std_residuals, c='green', alpha=0.7)
    ax2.axhline(y=0, color='r', linestyle='--')
    ax2.set_xlabel('Time from first expected arrival (hours)')
    ax2.set_ylabel('Standardized Residuals')
    ax2.set_title('Residuals Analysis')

    plt.tight_layout()
    plt.show()

    # Print detailed results
    print("\nWeighted Least Squares Regression Results:")
    print(f"Drift slope: {slope:.12f} s/s, i.e. {drift_ppm:.3f} ppm")
    print(f"95% Confidence Interval for slope: [{drift_ppm - ci_slope_ppm:.3f}:{drift_ppm + ci_slope_ppm:.3f}]")
    print(f"Drift offset at reference time: {offset:.3f} s, i.e. {offset_us:.3f} μs")

    # Residual Diagnostics
    print("\nResidual Diagnostics:")
    print(f"Shapiro-Wilk Test p-value (test for normality): {shapiro_p:.4f}")

    return {
        'slope': slope,
        'drift_ppm': drift_ppm,
        'offset': offset,
        'ci_slope_ppm': ci_slope_ppm,
        'shapiro_p': shapiro_p
    }

# Example usage
results = enhanced_drift_analysis(df)


Weighted Least Squares Regression Results:
Drift slope: 0.000000064781 s/s, i.e. 0.065 ppm
95% Confidence Interval for slope: [-0.144:0.274]
Drift offset at reference time: 0.157 s, i.e. 157433.596 μs

Residual Diagnostics:
Shapiro-Wilk Test p-value (test for normality): 0.4592
